# Installation 

Run **pip3 install -e .** in the folder with the setup.py

# Example Run

In [33]:
import tensorflow as tf
from e2ebench import Benchmark,\
                     ConfusionMatrixTracker,\
                     HyperparameterTracker,\
                     BenchmarkSupervisor,\
                     TimeMetric,\
                     MemoryMetric,\
                     PowerMetric,\
                     EnergyMetric,\
                     ThroughputMetric,\
                     LatencyMetric,\
                     TTATracker,\
                     LossTracker,\
                     CPUMetric
from sklearn.metrics import confusion_matrix
from keras import optimizers

def prep_data():
    fashion_mnist = tf.keras.datasets.fashion_mnist

    (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
    class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
                   'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

    train_images = train_images / 255.0
    test_images = test_images / 255.0

    return train_images, test_images, train_labels, test_labels, class_names

In [34]:
bm = Benchmark('sample_db_file.db', description="description")

bloat_metrics = {
    "throughput": ThroughputMetric('bloat throughput'),
    "latency": LatencyMetric('bloat latency'),
    "time": TimeMetric('bloat time'),
    "memory": MemoryMetric('bloat memory', interval=0.1),
    "power": PowerMetric('bloat power'),
    "energy": EnergyMetric('bloat energy'),
    "cpu": CPUMetric('bloat cpu', interval=0.1)
}

In [42]:
@BenchmarkSupervisor(bloat_metrics.values(), bm)
def train(train_images, train_labels):
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10)
    ])

    model.compile(optimizer="Adam",
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    history = model.fit(train_images, train_labels, epochs=10, batch_size=256)

    bloat_metrics["throughput"].track(num_entries=len(train_images))
    bloat_metrics["latency"].track(num_entries=len(train_images))

    TTATracker(bm).track(accuracies=history.history["accuracy"],  description="clothing TTA")
    LossTracker(bm).track(loss_values=history.history['loss'], description="clothing loss")
    
    return model

In [ ]:
def test(model, test_images, test_labels, class_names):

    model.evaluate(test_images, test_labels)
    pred_test = model.predict_classes(test_images)
    con_mat = confusion_matrix(test_labels, pred_test)
    ConfusionMatrixTracker(bm).track(con_mat, class_names, "confusion matrix")
    
    bm.close()

In [43]:
train_images, test_images, train_labels, test_labels, class_names = prep_data()
model = train(train_images, train_labels)
test(model, test_images, test_labels, class_names)

2021-07-26 13:40:53.128730: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 188160000 exceeds 10% of free system memory.


Epoch 1/10
235/235 [==============================] - 1s 3ms/step - loss: 0.8690 - accuracy: 0.7114
Epoch 2/10
235/235 [==============================] - 1s 3ms/step - loss: 0.4414 - accuracy: 0.8489
Epoch 3/10
235/235 [==============================] - 1s 3ms/step - loss: 0.3959 - accuracy: 0.8628
Epoch 4/10
235/235 [==============================] - 1s 3ms/step - loss: 0.3678 - accuracy: 0.8720
Epoch 5/10
235/235 [==============================] - 1s 3ms/step - loss: 0.3469 - accuracy: 0.8776
Epoch 6/10
235/235 [==============================] - 1s 3ms/step - loss: 0.3349 - accuracy: 0.8809
Epoch 7/10
235/235 [==============================] - 1s 3ms/step - loss: 0.3165 - accuracy: 0.8867
Epoch 8/10
235/235 [==============================] - 1s 3ms/step - loss: 0.3068 - accuracy: 0.8899
Epoch 9/10
235/235 [==============================] - 1s 3ms/step - loss: 0.3026 - accuracy: 0.8914
Epoch 10/10
235/235 [==============================] - 1s 3ms/step - loss: 0.2821 - accuracy: 0.8985

# Visualization

In der CLI ausführen: **umlaut-cli sample_db_file.db**

![title](Schreibtisch/Figure_1.png)
![title](Schreibtisch/Figure_2.png)
![title](Schreibtisch/Figure_3.png)
![title](Schreibtisch/Figure_4.png)
![title](Schreibtisch/Figure_5.png)
![title](Schreibtisch/Figure_6.png)
![title](Schreibtisch/Figure_7.png)
![title](Schreibtisch/Figure_8.png)
![title](Schreibtisch/Figure_9.png)
![title](Schreibtisch/Figure_10.png)

# Hyperparameter Tracking

In [45]:
def train(train_images, train_labels):
    ht = HyperparameterTracker(bm, "hyper params of sample pipeline", ['lr', 'num_epochs', 'batch_size'],
                               'loss')
    for lr in [0.1, 0.01, 0.001]:
        for num_epochs in [5, 10, 20]:
            for batch_size in [64, 128, 256]:

                model = tf.keras.Sequential([
                    tf.keras.layers.Flatten(input_shape=(28, 28)),
                    tf.keras.layers.Dense(128, activation='relu'),
                    tf.keras.layers.Dense(10)
                ])

                optimizer = optimizers.Adam(lr=lr)

                model.compile(optimizer=optimizer,
                              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                              metrics=['accuracy'])

                history = model.fit(train_images, train_labels, epochs=num_epochs, batch_size=batch_size)
                ht.track({'lr': lr, 'num_epochs': num_epochs, 'batch_size': batch_size, 'loss': history.history["accuracy"][-1]})
    ht.close()

![title](Schreibtisch/Figure_11.png)

# Troubleshooting 

## PyRAPL (Intel) - No Access

Run **sudo chmod -R a+r /sys/class/powercap/intel-rapl**